In [ ]:
!python --version

Python 3.11.11


In [1]:
import numpy as np


**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [1]:
class Module(object):
    """
    Basically, you can think of a module as of a something (black box)
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`:

        output = module.forward(input)

    The module should be able to perform a backward pass: to differentiate the `forward` function.
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule.

        gradInput = module.backward(input, gradOutput)
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True

    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)

    def backward(self,input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.

        This includes
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput


    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.

        Make sure to both store the data in `output` field and return it.
        """

        # The easiest case:

        # self.output = input
        # return self.output

        pass

    def updateGradInput(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own input.
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.

        The shape of `gradInput` is always the same as the shape of `input`.

        Make sure to both store the gradients in `gradInput` field and return it.
        """

        # The easiest case:

        # self.gradInput = gradOutput
        # return self.gradInput

        pass

    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass

    def zeroGradParameters(self):
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass

    def getParameters(self):
        """
        Returns a list with its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def train(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True

    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [3]:
class Sequential(Module):
    """
    This class implements a container, which processes `input` data sequentially.

    `input` is processed by each module (layer) in self.modules consecutively.
    The resulting array is called `output`.
    """

    def __init__(self):
        super(Sequential, self).__init__()
        self.modules = []
        self.prev_outputs = []

    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)

    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:

            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})

        Just write a little loop."""

        self.prev_outputs = []
        self.output = input

        for module in self.modules:
            self.prev_outputs.append(self.output)
            self.output = module.forward(self.output)
        self.output = self.output
        return self.output

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:

            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)
            gradInput = module[0].backward(input, g_1)

        !!!

        To each module you need to provide the input, module saw while forward pass,
        it is used while computing gradients.
        Make sure that the input for `i-th` layer is the output of `module[i]` (just the same input as in forward pass)
        and NOT `input` to this Sequential module.

        !!!
        """

        next_grad_output = gradOutput
        for i in range(len(self.modules) - 1, -1, -1):
            module = self.modules[i]
            prev_output = self.prev_outputs[i]
            next_grad_output = module.backward(prev_output, next_grad_output)
        self.gradInput = next_grad_output
        return self.gradInput



    def zeroGradParameters(self):
        for module in self.modules:
            module.zeroGradParameters()

    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]


    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]

    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string

    def __getitem__(self, x):
        return self.modules.__getitem__(x)

    def train(self):
        """
        Propagates training parameter through all modules
        """
        self.training = True
        for module in self.modules:
            module.train()

    def evaluate(self):
        """
        Propagates training parameter through all modules
        """
        self.training = False
        for module in self.modules:
            module.evaluate()

# Layers

## 1 (0.2). Linear transform layer
Also known as dense layer, fully-connected layer, FC-layer, InnerProductLayer (in caffe), affine transform
- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [205]:
class Linear(Module):
    """
    A module which applies a linear transformation
    A common name is fully-connected layer, InnerProductLayer in caffe.

    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()

        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size = n_out)

        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

    def updateOutput(self, input):

        self.output = np.dot(input,self.W.T)+ self.b #y = wx+b
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput =np.dot(gradOutput,self.W) #dL/dx = dL/dy*dy/dx
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        self.gradW=np.dot(gradOutput.T,input) #dL/dw = dL/dy* dy/dw - x
        self.gradb = np.sum(gradOutput, axis=0)  #dL/db = dL/dy* dy/db - 1
        return self.gradW, self.gradb

    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

## 2. (0.2) SoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$

Recall that $\text{softmax}(x) == \text{softmax}(x - \text{const})$. It makes possible to avoid computing exp() from large argument.

In [ ]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()

    def updateOutput(self, input):
      input = np.array(input,copy=True)
      self.output = np.subtract(input, input.max(axis=1, keepdims=True))
      self.output = np.exp(self.output) / np.sum(np.exp(self.output), axis=1, keepdims=True)
      return self.output


    def updateGradInput(self, input, gradOutput):
        input = np.array(input,copy=True)
        gradOutput =  np.array(gradOutput,copy=True)
        batch_size = input.shape[0]

        self.gradInput = np.zeros_like(input)
        for i in range(batch_size):
            x = self.output[i].reshape(-1, 1)
            grad = gradOutput[i]

            jacobian = np.diagflat(x) - np.dot(x, x.T)

            self.gradInput[i] = np.dot(jacobian, grad)
        return self.gradInput

    def __repr__(self):
      return "SoftMax"

## 3. (0.2) LogSoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{logsoftmax}(x)_i = \log\text{softmax}(x)_i = x_i - \log {\sum_j \exp x_j}$

The main goal of this layer is to be used in computation of log-likelihood loss.

In [ ]:
class LogSoftMax(Module):
    def __init__(self):
         super(LogSoftMax, self).__init__()

    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        self.output = self.output - np.log(np.sum(np.exp(self.output), axis=1,keepdims=True))
        return self.output

    def updateGradInput(self, input, gradOutput):
        softmax_output = np.exp(self.output)/np.sum(np.exp(self.output), axis=1,keepdims=True)
        self.gradInput = gradOutput - softmax_output * np.sum(gradOutput, axis=1, keepdims=True)
        return self.gradInput

    def __repr__(self):
        return "LogSoftMax"

## 4. (0.3) Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ - mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance:
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance.

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

In [ ]:
class BatchNormalization(Module):
    EPS = 1e-3
    def __init__(self, alpha = 0.):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = None
        self.moving_variance = None

    def updateOutput(self, input):
       if self.moving_mean is None:
        self.moving_mean = np.zeros(input.shape[1])
        self.moving_variance = np.ones(input.shape[1])

       if self.training == True:
        mean = np.mean(input,axis=0) #среднее и дисперсия по батчу
        variance = np.var(input,axis = 0)
        self.moving_mean = self.moving_mean * self.alpha + mean * (1 - self.alpha)
        self.moving_variance = self.moving_variance * self.alpha + variance * (1 - self.alpha)

       else:
        mean = self.moving_mean
        variance = self.moving_variance

       self.output = (input - mean) / np.sqrt(variance + self.EPS)
       return self.output

    def updateGradInput(self, input, gradOutput):
      if self.training:
        mean = np.mean(input, axis=0)
        variance = np.var(input, axis=0)
        std_inv = 1.0 / np.sqrt(variance + self.EPS)

        grad_normalized = gradOutput * std_inv # = dL/d(xnorm) * d(xnorm)/dx

        # Градиент для дисперсии
        grad_variance = np.sum(
            gradOutput * (input - mean) * -0.5 * (variance + self.EPS) ** (-1.5),
            axis=0
        ) # dL/d(var) = dL/d(xnorm)*d(xnorm)/d(var)

        # Градиент для среднего
        grad_mean = np.sum(gradOutput * -std_inv, axis=0) + grad_variance * np.mean(
            -2.0 * (input - mean), axis=0
        ) #dL/d(mean) = dL/d(xnorm) * d(xnorm)/d(mean) + dL/d(var) * d(var)/d(mean)

        # Итоговый градиент по входным данным - dL/dx = dL/dxnorm * dxnorm/dx + dL/d(var)*d(var)/dx + dL/d(mean)*d(mean)/dx
        self.gradInput = (
            grad_normalized  # Вклад от нормализованных данных
            + grad_variance * 2.0 * (input - mean) / input.shape[0]  # Вклад от дисперсии
            + grad_mean / input.shape[0]  # Вклад от среднего
        )
      else:
          self.gradInput = gradOutput / np.sqrt(self.moving_variance + self.EPS)

      return self.gradInput

    def __repr__(self):
        return "BatchNormalization"

In [ ]:
class ChannelwiseScaling(Module):
    """
       Implements linear transform of input y = \gamma * x + \beta
       where \gamma, \beta - learnable vectors of length x.shape[-1]
    """
    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()

        stdv = 1./np.sqrt(n_out)
        self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
        self.beta = np.random.uniform(-stdv, stdv, size=n_out)

        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def updateOutput(self, input):
        self.output = input * self.gamma + self.beta
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * self.gamma
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        self.gradBeta = np.sum(gradOutput, axis=0)
        self.gradGamma = np.sum(gradOutput*input, axis=0)

    def zeroGradParameters(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)

    def getParameters(self):
        return [self.gamma, self.beta]

    def getGradParameters(self):
        return [self.gradGamma, self.gradBeta]

    def __repr__(self):
        return "ChannelwiseScaling"

Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.

## 5. (0.3) Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [ ]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()

        self.p = p
        self.mask = None

    def updateOutput(self, input):
      if self.training:
        self.mask = np.random.binomial(1, 1 - self.p, size=input.shape)
        self.mask=self.mask/(1-self.p)
        self.output = self.mask*input
      else:
        self.output=input

      return  self.output

    def updateGradInput(self, input, gradOutput):
      if self.training:
        self.gradInput = gradOutput * self.mask
      else:
        self.gradInput = gradOutput

      return self.gradInput


    def __repr__(self):
        return "Dropout"

#6. (2.0) Conv2d
Implement [**Conv2d**](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html). Use only this list of parameters: (in_channels, out_channels, kernel_size, stride, padding, bias, padding_mode) and fix dilation=1 and groups=1.

#7. (0.5) Implement [**MaxPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) and [**AvgPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html). Use only parameters like kernel_size, stride, padding (negative infinity for maxpool and zero for avgpool) and other parameters fixed as in framework.

In [ ]:
import numpy as np

class MaxPool2d(Module):
    def __init__(self, kernel_size, stride, padding):
        super(MaxPool2d, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

    def updateOutput(self, input):
        batch_size, in_channels, in_height, in_width = input.shape
        kernel_height, kernel_width = self.kernel_size, self.kernel_size if isinstance(self.kernel_size, int) else self.kernel_size
        stride_height, stride_width = self.stride, self.stride if isinstance(self.stride, int) else self.stride
        padding_height, padding_width = self.padding, self.padding if isinstance(self.padding, int) else self.padding


        out_height = (in_height - kernel_height + 2 * padding_height) // stride_height + 1
        out_width = (in_width - kernel_width + 2 * padding_width) // stride_width + 1

        padded_input = np.pad(input, ((0, 0), (0, 0), (padding_height, padding_height), (padding_width, padding_width)), mode='constant', constant_values=-np.inf)

        self.output = np.zeros((batch_size, in_channels, out_height, out_width))

        for b in range(batch_size):
            for c in range(in_channels):
                for i in range(out_height):
                    for j in range(out_width):
                        h_start = i * stride_height
                        h_end = h_start + kernel_height
                        w_start = j * stride_width
                        w_end = w_start + kernel_width

                        self.output[b, c, i, j] = np.max(padded_input[b, c, h_start:h_end, w_start:w_end])

        return self.output

    def updateGradInput(self, input, gradOutput):
        batch_size, in_channels, in_height, in_width = input.shape
        kernel_height, kernel_width = self.kernel_size, self.kernel_size if isinstance(self.kernel_size, int) else self.kernel_size
        stride_height, stride_width = self.stride, self.stride if isinstance(self.stride, int) else self.stride
        padding_height, padding_width = self.padding, self.padding if isinstance(self.padding, int) else self.padding


        out_height = (in_height - kernel_height + 2 * padding_height) // stride_height + 1
        out_width = (in_width - kernel_width + 2 * padding_width) // stride_width + 1

        padded_input = np.pad(input, ((0, 0), (0, 0), (padding_height, padding_height), (padding_width, padding_width)), mode='constant', constant_values=-np.inf)

        self.gradInput = np.zeros_like(input)

        for b in range(batch_size):
            for c in range(in_channels):
                for i in range(out_height):
                    for j in range(out_width):
                        h_start = i * stride_height
                        h_end = h_start + kernel_height
                        w_start = j * stride_width
                        w_end = w_start + kernel_width

                        window = padded_input[b, c, h_start:h_end, w_start:w_end]
                        max_val = np.max(window)
                        mask = (window == max_val)
                        self.gradInput[b, c, h_start:h_end, w_start:w_end] += mask * gradOutput[b, c, i, j]

        return self.gradInput

    def __repr__(self):
        return "MaxPool2d"



class AvgPool2d(Module):
    def __init__(self, kernel_size, stride=None, padding=0):
        super(AvgPool2d, self).__init__()
        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.stride = stride if stride is not None else kernel_size
        self.stride = self.stride if isinstance(self.stride, tuple) else (self.stride, self.stride)
        self.padding = padding if isinstance(padding, tuple) else (padding, padding)

    def updateOutput(self, input):
        batch_size, in_channels, in_height, in_width = input.shape
        kernel_height, kernel_width = self.kernel_size
        stride_height, stride_width = self.stride
        padding_height, padding_width = self.padding

        out_height = (in_height + 2 * padding_height - kernel_height) // stride_height + 1
        out_width = (in_width + 2 * padding_width - kernel_width) // stride_width + 1


        padded_input = np.pad(
            input,
            ((0, 0), (0, 0), (padding_height, padding_height), (padding_width, padding_width)),
            mode='constant',
            constant_values=0
        )


        self.output = np.zeros((batch_size, in_channels, out_height, out_width))


        for b in range(batch_size):
            for c in range(in_channels):
                for i in range(out_height):
                    for j in range(out_width):
                        h_start = i * stride_height
                        h_end = h_start + kernel_height
                        w_start = j * stride_width
                        w_end = w_start + kernel_width


                        window = padded_input[b, c, h_start:h_end, w_start:w_end]

                        self.output[b, c, i, j] = np.mean(window)

        return self.output

    def updateGradInput(self, input, gradOutput):
        batch_size, in_channels, in_height, in_width = input.shape
        kernel_height, kernel_width = self.kernel_size
        stride_height, stride_width = self.stride
        padding_height, padding_width = self.padding


        out_height = (in_height + 2 * padding_height - kernel_height) // stride_height + 1
        out_width = (in_width + 2 * padding_width - kernel_width) // stride_width + 1


        padded_input = np.pad(
            input,
            ((0, 0), (0, 0), (padding_height, padding_height), (padding_width, padding_width)),
            mode='constant',
            constant_values=0
        )

        self.gradInput = np.zeros_like(padded_input)

        for b in range(batch_size):
            for c in range(in_channels):
                for i in range(out_height):
                    for j in range(out_width):
                        h_start = i * stride_height
                        h_end = h_start + kernel_height
                        w_start = j * stride_width
                        w_end = w_start + kernel_width


                        grad_value = gradOutput[b, c, i, j] / (kernel_height * kernel_width)
                        self.gradInput[b, c, h_start:h_end, w_start:w_end] += grad_value


        if padding_height > 0 or padding_width > 0:
            self.gradInput = self.gradInput[
                :, :,
                padding_height:padding_height + in_height,
                padding_width:padding_width + in_width
            ]

        return self.gradInput

    def __repr__(self):
        return "AvgPool2d"

#8. (0.3) Implement **GlobalMaxPool2d** and **GlobalAvgPool2d**. They do not have testing and parameters are up to you but they must aggregate information within channels. Write test functions for these layers on your own.

In [ ]:
class GlobalMaxPool2d(Module):
    def __init__(self):
        super(GlobalMaxPool2d, self).__init__()
        self.cache = None
    def updateOutput(self, x):
      self.cache = x
      out =  np.max(x.reshape(x.shape[0], x.shape[1], -1), axis=2)

      return out
    def updateGradInput(self, dout):
        x = self.cache
        batch_size, channels, height, width = x.shape

        x_reshaped = x.reshape(batch_size, channels, -1)
        max_vals = np.max(x_reshaped, axis=2, keepdims=True)

        mask = (x_reshaped == max_vals)

        dout_reshaped = dout[..., np.newaxis] * mask
        dx = dout_reshaped.reshape(batch_size, channels, height, width)

        return dx


In [ ]:
class GlobalAvgPool2d(Module):

    def __init__(self):
        self.cache = None

    def updateOutput(self, x):

        self.input_shape = x.shape
        self.cache = x


        return np.mean(x, axis=(2,3))

    def updateGradInput(self, dout):

        batch_size, channels, height, width = self.input_shape

        scale = 1 / (height * width)
        dx = np.ones(self.input_shape) * dout[:, :, np.newaxis, np.newaxis] * scale

        return dx

#9. (0.2) Implement [**Flatten**](https://pytorch.org/docs/stable/generated/torch.flatten.html)

In [ ]:
class Flatten(Module):

    def __init__(self, start_dim=0, end_dim=-1):
        super(Flatten, self).__init__()
        self.start_dim = start_dim
        self.end_dim = end_dim

    def updateOutput(self, input):

        self.input_shape = input.shape

        if self.end_dim == -1:
            self.end_dim = len(input.shape) - 1

        flattened_size = 1
        for i in range(self.start_dim, self.end_dim + 1):
            flattened_size *= input.shape[i]

        new_shape = list(input.shape[:self.start_dim]) + [flattened_size] + list(input.shape[self.end_dim + 1:])

        self.output = input.reshape(*new_shape)
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput.reshape(self.input_shape)
        return self.gradInput

    def __repr__(self):
        return "Flatten"


# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**):

In [9]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()

    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput

    def __repr__(self):
        return "ReLU"

## 10. (0.1) Leaky ReLU
Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope.

In [ ]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super(LeakyReLU, self).__init__()

        self.slope = slope

    def updateOutput(self, input):
        self.output=np.where(input > 0, input, self.slope* input)
        return  self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput=np.multiply(gradOutput, np.where(input > 0, 1, self.slope))
        return self.gradInput

    def __repr__(self):
        return "LeakyReLU"

## 11. (0.1) ELU
Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

In [ ]:
class ELU(Module):
    def __init__(self, alpha = 1.0):
        super(ELU, self).__init__()

        self.alpha = alpha

    def updateOutput(self, input):
       self.output=np.where(input > 0, input, self.alpha* (np.exp(input)-1))
       return  self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput=np.multiply(gradOutput, np.where(input > 0, 1, self.alpha*np.exp(input)))
        return self.gradInput

    def __repr__(self):
        return "ELU"

## 12. (0.1) SoftPlus
Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

In [ ]:
class SoftPlus(Module):
    def __init__(self, beta=1.0,threshold=20.0):
        super(SoftPlus, self).__init__()
        self.beta=beta
        self.threshold=threshold

    def updateOutput(self, input):
        self.output=np.where(input > self.threshold/self.beta, input, 1/(self.beta)* np.log(1+np.exp(self.beta*input)))
        return  self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput=np.multiply(gradOutput, np.where(input > self.threshold/self.beta, 1, np.exp(self.beta*input)/(np.exp(self.beta*input)+1)))
        return self.gradInput

    def __repr__(self):
        return "SoftPlus"

#13. (0.2) Gelu
Implement [**Gelu**](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html) activations.

In [ ]:
import torch
import torch.nn as nn
import math

class Gelu(Module):
    def __init__(self):
        super(Gelu, self).__init__()

    def updateOutput(self, input):
        input = torch.tensor(input,requires_grad=True)

        self.output =  0.5 * input * (1 + torch.erf(input / math.sqrt(2)))
        return self.output.detach().numpy()

    def updateGradInput(self, input, gradOutput):
        input = torch.tensor(input,requires_grad=True)
        gradOutput = torch.tensor(gradOutput,requires_grad=True)

        cdf = 0.5 * (1 + torch.erf(input / math.sqrt(2)))
        pdf = torch.exp(-0.5 * input**2) / math.sqrt(2 * math.pi)
        self.gradInput = gradOutput * (cdf + input * pdf)
        return self.gradInput.detach().numpy()

    def __repr__(self):
        return "Gelu"

# Criterions

In [10]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None

    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)

    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.
- input:   **`batch_size x n_feats`**
- target: **`batch_size x n_feats`**
- output: **scalar**

In [11]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()

    def updateOutput(self, input, target):
        self.output = np.sum(np.power(input - target,2),axis = 0) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

## 14. (0.2) Negative LogLikelihood criterion (numerically unstable)
You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula,
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. Also there is a small hack with adding small number to probabilities to avoid computing log(0).
- input:   **`batch_size x n_feats`** - probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**



In [ ]:
class ClassNLLCriterionUnstable(Criterion):
    EPS = 1e-15
    def __init__(self):
        a = super(ClassNLLCriterionUnstable, self)
        super(ClassNLLCriterionUnstable, self).__init__()

    def updateOutput(self, input, target):

        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        self.output = -np.sum(target * np.log(input_clamp)) / input_clamp.shape[0]
        return self.output

    def updateGradInput(self, input, target):

        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        self.gradInput = -target / input_clamp / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterionUnstable"

## 15. (0.3) Negative LogLikelihood criterion (numerically stable)
- input:   **`batch_size x n_feats`** - log probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

Task is similar to the previous one, but now the criterion input is the output of log-softmax layer. This decomposition allows us to avoid problems with computation of forward and backward of log().

In [ ]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).__init__()

    def updateOutput(self, input, target):
       self.output = -np.sum(target * input) / input.shape[0]
       return self.output

    def updateGradInput(self, input, target):
      self.gradInput = -target / input.shape[0]
      return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterion"

1-я часть задания: реализация слоев, лосей и функций активации - 5 баллов. \\
2-я часть задания: реализация моделей на своих классах. Что должно быть:
  1. Выберите оптимизатор и реализуйте его, чтоб он работал с вами классами. - 1 балл.
  2. Модель для задачи мультирегрессии на выбраных вами данных. Использовать FCNN, dropout, batchnorm, MSE. Пробуйте различные фукнции активации. Для первой модели попробуйте большую, среднюю и маленькую модель. - 1 балл.
  3. Модель для задачи мультиклассификации на MNIST. Использовать свёртки, макспулы, флэттэны, софтмаксы - 1 балла.
  4. Автоэнкодер для выбранных вами данных. Должен быть на свёртках и полносвязных слоях, дропаутах, батчнормах и тд. - 2 балла. \\

Дополнительно в оценке каждой модели будет учитываться:
1. Наличие правильно выбранной метрики и лосс функции.
2. Отрисовка графиков лосей и метрик на трейне-валидации. Проверка качества модели на тесте.
3. Наличие шедулера для lr.
4. Наличие вормапа.
5. Наличие механизма ранней остановки и сохранение лучшей модели.
6. Свитч лося (метрики) и оптимайзера.

In [204]:
class SGD(Module):
    def __init__(self, model, lr=0.01, momentum=0.0, weight_decay=0.0):

        self.model = model
        self.lr = lr
        self.momentum = momentum
        self.weight_decay = weight_decay
        self.velocities = []

        params = self._flatten_parameters(self.model.getParameters())
        for param in params:
            self.velocities.append(np.zeros_like(param))

    def _flatten_parameters(self, parameters):

        flat_params = []
        for module_params in parameters:
            for param in module_params:
                flat_params.append(param)
        return flat_params

    def _flatten_gradients(self, gradients):

        flat_grads = []
        for module_grads in gradients:
            for grad in module_grads:
                flat_grads.append(grad)
        return flat_grads

    def step(self):

        params = self._flatten_parameters(self.model.getParameters())
        grads = self._flatten_gradients(self.model.getGradParameters())

        for i, (param, grad) in enumerate(zip(params, grads)):
            if self.weight_decay != 0:
                grad += self.weight_decay * param

            if self.momentum != 0:
                self.velocities[i] = self.momentum * self.velocities[i] + self.lr * grad
                param -= self.velocities[i]
            else:
                param -= self.lr * grad

    def zero_grad(self):

        self.model.zeroGradParameters()